Define the brickcomps 

In [1]:
using DataFrames
using CSV
AIS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_antarctic_RCP85_sneasybrick.csv") |> DataFrame;

In [2]:
GMSL_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_gmsl_RCP85_sneasybrick.csv") |> DataFrame;

In [3]:
LWS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_landwater_storage_sl_RCP85_sneasybrick.csv") |> DataFrame;

In [4]:
TE_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_thermal_RCP85_sneasybrick.csv") |> DataFrame;

In [5]:
GIS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_greenland_RCP85_sneasybrick.csv") |> DataFrame;

In [6]:
GSIC_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_glaciers_RCP85_sneasybrick.csv") |> DataFrame;

Construct Brick components

In [7]:
Sim=10000
GMSL=GMSL_Data[:,1:Sim];
LWS=LWS_Data[:,1:Sim];
GIS= GIS_Data[:,1:Sim];
GSIC=GSIC_Data[:,1:Sim];
TE=TE_Data[:,1:Sim];
AIS=AIS_Data[:,1:Sim];

In [8]:
Y=collect(1850:2300);

btime=Y;



In [9]:
brickcomps=Tuple([btime,AIS,GSIC,GIS,TE,LWS,GMSL]);

Run Brick downscale function

In [10]:
function get_fingerprints()

    fp_dir = joinpath(@__DIR__, "..", "data")
    isdir(fp_dir) || mkpath(fp_dir)
    fp_file = joinpath(fp_dir, "FINGERPRINTS_SLANGEN_Bakker.nc")
    if !isfile(fp_file)
        url = "https://github.com/scrim-network/BRICK/raw/master/fingerprints/FINGERPRINTS_SLANGEN_Bakker.nc"
        download(url, fp_file)
    end

    fplat = ncread(fp_file,"lat")
    fplon = ncread(fp_file,"lon")
    fpAIS = ncread(fp_file,"AIS")
    fpGSIC = ncread(fp_file,"GLAC")
    fpGIS = ncread(fp_file,"GIS")
    ncclose()

    return fplat,fplon,fpAIS,fpGSIC,fpGIS
end

get_fingerprints (generic function with 1 method)

In [11]:
function subtractor(minval,maxval)
    function y(point,n)
        if point - n < minval
            return min(maxval,point - n + maxval)
        else
            return point - n
        end
    end
end

subtractor (generic function with 1 method)

In [12]:
function adder(maxval)
    function y(point,n)
        if point + n > maxval
            return point + n - maxval
        else
            return point + n
        end
    end
end

adder (generic function with 1 method)

In [13]:
lon_subtractor = subtractor(1,360)
lon_adder = adder(360)
lat_adder = adder(180)
lat_subtractor = subtractor(1,180)

(::var"#y#1"{Int64, Int64}) (generic function with 1 method)

In [14]:
using Statistics
using NetCDF
function downscale_brick(brickcomps, lonlat, ensInds, ystart=2010, yend=2100, tstep=10)
    # To do - check with vectors of lat, lon
    (fplat,fplon,fpAIS,fpGSIC,fpGIS) = get_fingerprints()
    (btime,AIS,GSIC,GIS,TE,LWS,GMSL) = brickcomps

    # Select indices of time of interest, with respect to timestep
    tinds = findall(x -> x .>= ystart && x .<=yend, btime)
    years = collect(ystart:yend)
    yinds = findall(x -> x % tstep==0, years)
    # Need to normalize LSL relative to 2000
    inorm = findall(x -> x==2000, btime)

    tdim=length(btime)

    if length(years)==length(tinds)
        tinds = tinds[yinds]
    else
        println("Error: years outside of bounds")
        return nothing
    end

    num_ens = length(ensInds)
    
    # Output matrix: ens x time x segment
    lsl_out = zeros(num_ens, length(tinds), length(lonlat))

    # Trim component vectors to timesteps and ensembles. Assume interval is 1 year
    if tdim==size(AIS)[1] # check that time dimension is 1
        # for normalizing
        AIS_norm = AIS[inorm,ensInds]
        GSIC_norm = GSIC[inorm,ensInds]
        GIS_norm = GIS[inorm,ensInds]
        TE_norm = TE[inorm,ensInds]
        LWS_norm = LWS[inorm,ensInds]
        GMSL_norm = GMSL[inorm,ensInds]
        # actual projections
        AIS = AIS[tinds,ensInds]
        GSIC = GSIC[tinds,ensInds]
        GIS = GIS[tinds,ensInds]
        TE = TE[tinds,ensInds]
        LWS = LWS[tinds,ensInds]
        GMSL = GMSL[tinds,ensInds]
    else
        println("Error: time dimension is not 1 for brick components")
        return nothing
    end

  

    for f in 1:length(lonlat) # Loop through lonlat tuples

        lon = lonlat[f][1]
        lat = lonlat[f][2]
        # Convert Longitude to degrees East
        # CIAM Lat is already in (-90,90) by default
        if lon <0
            lon = lon + 360
        end

        # Find fingerprint degrees nearest to lat,lon
        ilat = findall(isequal(minimum(abs.(fplat.-lat))),abs.(fplat.-lat))
        ilon = findall(isequal(minimum(abs.(fplon.-lon))),abs.(fplon.-lon))


        # Take average of closest lat/lon values
        fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[ilon,ilat])))
        fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[ilon,ilat])))
        fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[ilon,ilat])))

        fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
        fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
        fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]
        fpTE_loc = 1.0
        fpLWS_loc=1.0

        # Keep searching nearby lat/lon values if fingerprint value is NaN unless limit is hit
        inc =1

        while isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc) && inc<5

            newlonStart = lon_subtractor.(fplon[ilon],inc)[1]
            newlatStart = lat_subtractor.(fplat[ilat],inc)[1]
            newlonEnd = lon_adder.(fplon[ilon],inc)[1]
            newlatEnd = lat_adder.(fplat[ilat],inc)[1]

            latInd1 = minimum(findall(isequal(minimum(abs.(fplat.-newlatStart))),abs.(fplat.-newlatStart)))
            #minimum(findall(x-> x in newlatStart,fplat))
            latInd2 = maximum(findall(isequal(minimum(abs.(fplat.-newlatEnd))),abs.(fplat.-newlatEnd)))
            #maximum(findall(x -> x in newlatEnd,fplat))

            lonInd1 = minimum(findall(isequal(minimum(abs.(fplon.-newlonStart))),abs.(fplon.-newlonStart)))
            #minimum(findall(x-> x in newlonStart,fplon))
            lonInd2 = maximum(findall(isequal(minimum(abs.(fplon.-newlonEnd))),abs.(fplon.-newlonEnd)))
            #maximum(findall(x -> x in newlonEnd,fplon))

            if latInd2 < latInd1
                latInds=[latInd1; 1:latInd2]
            else
                latInds=latInd1:latInd2
            end

            if lonInd2 < lonInd1
                lonInds=[lonInd1; 1:lonInd2]
            else
                lonInds = lonInd1:lonInd2
            end

            fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[lonInds,latInds])))
            fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[lonInds,latInds])))
            fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[lonInds,latInds])))

            fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
            fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
            fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]

            inc = inc + 1

        end

        # If still NaN, throw an error
        if isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc)
            println("Error: no fingerprints found for ($(lon),$(lat))")
            return nothing
        end
       # Multiply fingerprints by BRICK ensemble members
       if ndims(AIS) > 1
        for n in 1:size(AIS)[2] # loop through ensemble members
            lsl_out[n, :, f] = fpGIS_loc * GIS[:,n] + fpAIS_loc * AIS[:,n] + fpGSIC_loc * GSIC[:,n] +
                               fpTE_loc * TE[:,n] + fpLWS_loc * LWS[:,n]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm[1,n] + fpAIS_loc * AIS_norm[1,n] + fpGSIC_loc * GSIC_norm[1,n] +
                       fpTE_loc * TE_norm[1,n] + fpLWS_loc * LWS_norm[1,n]
            lsl_out[n, :, f] = lsl_out[n, :, f] .- lsl_norm
            
        end
    else
            lsl_out[1, :, f] = fpGIS_loc * GIS[:] + fpAIS_loc * AIS[:] + fpGSIC_loc * GSIC[:] +
                                fpTE_loc * TE[:] + fpLWS_loc * LWS[:]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm + fpAIS_loc * AIS_norm + fpGSIC_loc * GSIC_norm +
                                fpTE_loc * TE_norm + fpLWS_loc * LWS_norm
            lsl_out[1, :, f] = lsl_out[1, :, f] .- lsl_norm
    end

    end # End lonlat tuple

    return lsl_out,GMSL
end

downscale_brick (generic function with 4 methods)

In [ ]:
lonlat=([-87.39,30.47],[-87.961,30.675],[-87.917,30.604],[-87.911,30.562],[-87.923,30.521],[-87.944,30.227],[-88.156,30.324]
-88.063	30.615]
-88.031	30.679]
-88.443	30.329]
-89.418	30.212]
-89.26	30.319]
-89.173	30.335]
-89.061	30.366]
-88.97	30.418]
-88.946	30.417]
-88.884	30.415]
-88.809	30.394]
-88.728	30.348]
-88.625	30.361]
-88.562	30.349]
-88.528	30.338]
-89.645	30.169]
-90.124	30.023]
-90.372	30.245]
-90.082	30.364]
-89.958	30.26]
-89.648	29.271]
-89.938	30.064]
-89.803	30.218]
-92.941	29.706]
-93.846	29.986]
-91.914	29.645]
-85.88	30.221]
-85.725	30.133]
-85.834	30.288]
-85.681	30.207]
-85.649	30.147]
-85.613	30.13]
-85.592	30.121]
-85.563	30.108]
-85.409	30.034]
-85.574	30.088]
-85.097	29.715]
-87.348	30.426]
-87.492	30.284]
-87.24	30.375]
-87.179	30.434]
-87.155	30.503]
-86.941	30.456]
-87.159	30.368]
-86.962	30.395]
-86.807	30.413]
-86.723	30.419]
-86.66	30.417]
-86.624	30.415]
-86.589	30.42]
-86.549	30.441]
-86.508	30.462]
-86.459	30.486]
-86.229	30.388]
-87.272	30.321]
-87.187	30.343]
-86.959	30.379]
-86.771	30.404]
-86.688	30.409]
-86.625	30.411]
-86.528	30.397]
-86.625	30.4]
-86.688	30.399]
-86.771	30.395]
-86.959	30.364]
-87.109	30.337]
-87.193	30.325]
-83.203	29.399]
-81.789	26.703]
-81.813	26.673]
-81.842	26.636]
-81.863	26.61]
-81.896	26.568]
-81.922	26.511]
-81.957	26.498]
-81.967	26.485]
-81.935	26.468]
-81.889	26.449]
-81.855	26.382]
-81.831	26.271]
-81.82	26.188]
-81.806	26.12]
-81.758	26.02]
-81.722	25.949]
-81.151	25.422]
-82.584	27.546]
-82.588	27.521]
-82.624	27.478]
-82.603	27.421]
-82.558	27.354]
-82.545	27.319]
-82.533	27.277]
-82.514	27.229]
-82.496	27.188]
[-82.421	27.015]
[-82.166	26.785]
[-82.067	26.957]
[-82.036	26.896]
[-82.042	26.875]
[-82.08	26.702]
[-81.996	26.533]
[-81.938	26.563]
[-81.92	26.602]
[-81.916	26.623]
[-81.861	26.673]
[-81.789	26.706]
[-82.768	28.11]
[-82.845	27.909]
[-82.72	27.66]
[-82.621	27.728]
[-82.634	27.883]
[-82.702	27.981]
[-82.625	28.003]
[-82.563	27.978]
[-82.527	27.964]
[-82.485	27.948]
[-82.433	27.924]
[-82.397	27.847]
[-82.417	27.769]
[-82.543	27.646]
[-82.697	28.354]
[-82.707	28.329]
[-82.724	28.287]
[-82.746	28.239]
[-82.757	28.183]
[-97.295	26.097]
[-95.134	29.083]
[-94.998	29.544]
[-95.014	29.563]
[-95.027	29.604]
[-95.046	29.667]
[-95.054	29.715]
[-95.013	29.716]
[-95.895	28.641]
[-97.085	27.955]
[-97.032	28.02]
[-96.783	28.333]
[-96.609	28.611]
[-96.255	28.687]
[-97.123	27.909]
[-97.138	27.891]
[-97.21	27.821]
[-97.313	27.844]
[-97.466	27.873]
[-97.435	27.803]
[-97.358	27.753]
[-97.287	27.668]
[-97.553	27.239]
[-96.859	28.128]
[-96.415	28.393]
[-97.051	27.841]
[-97.385	27.197]
[-94.978	29.221]
[-94.988	29.175]
[-94.891	29.396]
[-94.906	29.431]
[-94.938	29.469]
[-94.977	29.518]
[-94.957	29.697]
[-94.923	29.711]
[-94.744	29.374]
[-93.935	29.854]
[-93.908	29.896]
[-93.885	29.933]
[-94.831	29.332]
[-94.82	29.338]
[-94.801	29.328]
[-94.783	29.315]
[-94.8	29.298]
[-94.821	29.283]
[-94.848	29.263])

In [15]:
ensInds=collect(1:10000);
lonlat= ([-122.4194,37.7749],[153.0260,-27.4705],[12.3155,45.4408],[4.3007,52.0705],[43.6159,-23.1453]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew85,H=downscale_brick(brickcomps, lonlat, ensInds, ystart, yend, tstep);

Save the Local SLR for Manzanillo, Colima and New Orleans in a CVS File 
1) Make a data frame for each one
2) Save it as a CVS File

RCP 8.5 case

In [18]:
#Random locations
RandomL85=zeros(5,20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for k in 1:5
    for i in 1:20
        RandomL85[k,i,1]=2000+i*10
    end
#SLR value for each year for each simulation 
    for i in 1:10000
        for j in 1:20
            RandomL85[k,j,i+1]=lslrNew85[i,j,k]
     end 
    end 
end

projections_Random_LSLR_RCP85_R=reshape(RandomL85,100,10001)

using CSV
using Tables
CSV.write("projections_Random_LSLR_RCP85.csv", Tables.table(projections_Random_LSLR_RCP85_R))

In [21]:
projections_Random_LSLR_RCP85_R=reshape(RandomL85,100,10001)

100×10001 Matrix{Float64}:
 2010.0  0.0236679  0.0228118  0.0313817  …  0.0277411  0.0299613  0.0282152
 2010.0  0.0281391  0.0243799  0.0376155     0.0330962  0.0323219  0.0309098
 2010.0  0.0230695  0.0213039  0.0313428     0.0269861  0.0281754  0.0266406
 2010.0  0.0217139  0.0201942  0.0302372     0.0255152  0.0269656  0.0254127
 2010.0  0.0282792  0.0246069  0.0375669     0.0332108  0.0325208  0.0310984
 2020.0  0.0588795  0.0557887  0.0663297  …  0.0599051  0.0649833  0.0623116
 2020.0  0.0718151  0.0656509  0.0787114     0.0690885  0.0740213  0.0721736
 2020.0  0.057702   0.053553   0.0649355     0.0566187  0.061881   0.0595175
 2020.0  0.0542664  0.0502594  0.0619169     0.0530837  0.0585722  0.0561419
 2020.0  0.0720709  0.0660362  0.0788563     0.0695712  0.0744422  0.0725703
 2030.0  0.1019     0.0917212  0.109307   …  0.1031     0.112156   0.10906
 2030.0  0.122903   0.109221   0.129123      0.120521   0.131768   0.129949
 2030.0  0.0971884  0.0874827  0.104962      0.09618

In [22]:
import Pkg 
Pkg.add("Tables")


┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36


In [23]:
using CSV
using Tables
CSV.write("projections_Random_LSLR_RCP85.csv", Tables.table(projections_Random_LSLR_RCP85_R))

"projections_Random_LSLR_RCP85.csv"

RCP 6.0 case

In [24]:
using DataFrames
using CSV
AIS_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_antarctic_RCP60_sneasybrick.csv") |> DataFrame;

In [25]:
GMSL_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_gmsl_RCP60_sneasybrick.csv") |> DataFrame;

In [26]:
LWS_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_landwater_storage_sl_RCP60_sneasybrick.csv") |> DataFrame;

In [27]:
TE_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_thermal_RCP60_sneasybrick.csv") |> DataFrame;

In [28]:
GIS_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_greenland_RCP60_sneasybrick.csv") |> DataFrame;

In [29]:
GSIC_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_glaciers_RCP60_sneasybrick.csv") |> DataFrame;

In [30]:
Sim=10000
GMSL60=GMSL_Data60[:,1:Sim];
LWS60=LWS_Data60[:,1:Sim];
GIS60= GIS_Data60[:,1:Sim];
GSIC60=GSIC_Data60[:,1:Sim];
TE60=TE_Data60[:,1:Sim];
AIS60=AIS_Data60[:,1:Sim];

In [31]:
brickcomps60=Tuple([btime,AIS60,GSIC60,GIS60,TE60,LWS60,GMSL60]);

In [32]:
ensInds=collect(1:10000);
lonlat= ([-122.4194,37.7749],[153.0260,-27.4705],[12.3155,45.4408],[4.3007,52.0705],[43.6159,-23.1453]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew60,H60=downscale_brick(brickcomps60, lonlat, ensInds, ystart, yend, tstep);

In [33]:
#Random locations
RandomL60=zeros(5,20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for k in 1:5
    for i in 1:20
        RandomL60[k,i,1]=2000+i*10
    end
#SLR value for each year for each simulation 
    for i in 1:10000
        for j in 1:20
            RandomL60[k,j,i+1]=lslrNew60[i,j,k]
     end 
    end 
end

projections_Random_LSLR_RCP60_R=reshape(RandomL60,100,10001)

using CSV
using Tables
CSV.write("projections_Random_LSLR_RCP60.csv", Tables.table(projections_Random_LSLR_RCP60_R))

"projections_Random_LSLR_RCP60.csv"

RCP 4.5 case

In [34]:
AIS_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_antarctic_RCP45_sneasybrick.csv") |> DataFrame;

In [35]:
GMSL_Data45= CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_gmsl_RCP45_sneasybrick.csv") |> DataFrame;

In [36]:
LWS_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_landwater_storage_sl_RCP45_sneasybrick.csv") |> DataFrame;

In [37]:
TE_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_thermal_RCP45_sneasybrick.csv") |> DataFrame;

In [38]:
GIS_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_greenland_RCP45_sneasybrick.csv") |> DataFrame;

In [39]:
GSIC_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_glaciers_RCP45_sneasybrick.csv") |> DataFrame;

In [40]:
Sim=10000
GMSL45=GMSL_Data45[:,1:Sim];
LWS45=LWS_Data45[:,1:Sim];
GIS45= GIS_Data45[:,1:Sim];
GSIC45=GSIC_Data45[:,1:Sim];
TE45=TE_Data45[:,1:Sim];
AIS45=AIS_Data45[:,1:Sim];

In [41]:
brickcomps45=Tuple([btime,AIS45,GSIC45,GIS45,TE45,LWS45,GMSL45]);

In [42]:
ensInds=collect(1:10000);
lonlat= ([-122.4194,37.7749],[153.0260,-27.4705],[12.3155,45.4408],[4.3007,52.0705],[43.6159,-23.1453]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew45,H=downscale_brick(brickcomps45, lonlat, ensInds, ystart, yend, tstep);

In [43]:
#Random locations
RandomL45=zeros(5,20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for k in 1:5
    for i in 1:20
        RandomL45[k,i,1]=2000+i*10
    end
#SLR value for each year for each simulation 
    for i in 1:10000
        for j in 1:20
            RandomL45[k,j,i+1]=lslrNew45[i,j,k]
     end 
    end 
end

projections_Random_LSLR_RCP45_R=reshape(RandomL45,100,10001)

using CSV
using Tables
CSV.write("projections_Random_LSLR_RCP45.csv", Tables.table(projections_Random_LSLR_RCP45_R))

"projections_Random_LSLR_RCP45.csv"

RCP 2.6 case

In [44]:
AIS_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_antarctic_RCP26_sneasybrick.csv") |> DataFrame;

In [45]:
GMSL_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_gmsl_RCP26_sneasybrick.csv") |> DataFrame;

In [46]:
LWS_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_landwater_storage_sl_RCP26_sneasybrick.csv") |> DataFrame;

In [47]:
TE_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_thermal_RCP26_sneasybrick.csv") |> DataFrame;

In [48]:
GIS_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_greenland_RCP26_sneasybrick.csv") |> DataFrame;

In [49]:
GSIC_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_glaciers_RCP26_sneasybrick.csv") |> DataFrame;

In [50]:
Sim=10000
GMSL26=GMSL_Data26[:,1:Sim];
LWS26=LWS_Data26[:,1:Sim];
GIS26= GIS_Data26[:,1:Sim];
GSIC26=GSIC_Data26[:,1:Sim];
TE26=TE_Data26[:,1:Sim];
AIS26=AIS_Data26[:,1:Sim];

In [51]:
brickcomps26=Tuple([btime,AIS26,GSIC26,GIS26,TE26,LWS26,GMSL26]);

In [52]:
ensInds=collect(1:10000);
lonlat= ([-122.4194,37.7749],[153.0260,-27.4705],[12.3155,45.4408],[4.3007,52.0705],[43.6159,-23.1453]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew26,H=downscale_brick(brickcomps26, lonlat, ensInds, ystart, yend, tstep);

In [53]:
#Random locations
RandomL26=zeros(5,20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for k in 1:5
    for i in 1:20
        RandomL26[k,i,1]=2000+i*10
    end
#SLR value for each year for each simulation 
    for i in 1:10000
        for j in 1:20
            RandomL26[k,j,i+1]=lslrNew26[i,j,k]
     end 
    end 
end

projections_Random_LSLR_RCP26_R=reshape(RandomL26,100,10001)

using CSV
using Tables
CSV.write("projections_Random_LSLR_RCP26.csv", Tables.table(projections_Random_LSLR_RCP26_R))

"projections_Random_LSLR_RCP26.csv"